In [ ]:
%matplotlib notebook
import math
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype']=42
matplotlib.rcParams['ps.fonttype']=42
import matplotlib.pyplot as plt
from ipywidgets import interact
plt.style.use('bmh')

# current implementation
def curve(x, g, sigma, shadows, highlights, clarity):
    c = x-g
    val = 0
    ssigma = - sigma
    if c > 0.0 : ssigma = sigma
    shadhi = highlights
    if c > 0.0 : shadhi = shadows
    if (abs(c) > 2*sigma):
        val = g + ssigma + shadhi * (c-ssigma) # linear part
    else:
        # blend in via quadratic bezier
        t = min(max(c / (2.0*ssigma), 0.0), 1.0)
        t2 = t * t
        mt = 1.0-t
        val = g + ssigma * 2.0*mt*t + t2*(ssigma + ssigma*shadhi)
    # midtone local contrast
    val += clarity * c * math.exp(-c*c/(2.0*sigma*sigma/3.0))
    return val

# derivative of gaussian
def curve2(x, g, sigma, shadows, highlights, clarity):
    return g + (x-g) + clarity*(x-g)/(sigma*sigma) \
        *1.0/(math.sqrt(2*3.141592)*sigma)*0.01 \
        *math.exp(-(x-g)*(x-g)/(2*sigma*sigma))

def sigmoid(x):
    return x/(2.*math.sqrt(1.+x*x))

def curve3(x, g, sigma, shadows, highlights, clarity):
    c = x - g
    s = clarity * 2
    c1 = shadows
    x1 = math.sqrt(max(0, pow(s,2./3.) - math.pow(2*c1,2./3.)))/(s*math.pow(2*c1,1./3.))
    c0 = highlights
    x0 = - math.sqrt(max(0, pow(s,2./3.) - math.pow(2*c0,2./3.)))/(s*math.pow(2*c0,1./3.))
    if(c < x0):
        return g + sigmoid(s*x0) + highlights * (c-x0)
    if(c > x1):
        return g + sigmoid(s*x1) + shadows * (c-x1)
    return g + sigmoid(s*c)

# define the curve via these control vertices here:
x=np.array([0.0,0.25,0.75,1.00])
y=np.array([0.0,0.62,0.14,0.97])

m=np.array([0.0,0.0,0.0,0.0,0.0])
d=np.array([0.0,0.0,0.0,0.0,0.0])


fig, ax = plt.subplots(figsize=(5,5))
ax.set_ylim([-0.5,1.5])
ax.set_xlim([-0.5,1.5])
pos = np.linspace(-0.5, 1.5, 100)
ax.axvline(x=0.0,ymin=0.0,ymax=1.0)
ax.axvline(x=1.0,ymin=0.0,ymax=1.0)
px1 = ax.axvline(x=x[1],ymin=0.0,ymax=1.0)
px2 = ax.axvline(x=x[2],ymin=0.0,ymax=1.0)
ax.axhline(y=0.0,xmin=0.0,xmax=1.0)
ax.axhline(y=1.0,xmin=0.0,xmax=1.0)
plot = ax.plot(pos, np.zeros(pos.shape), color='C1')[0]

@interact(g=.5,s=1.,h=1.,c=1.,sig=0.2)
def test_spline(g,s,h,c,sig):
    # plot.set_ydata(np.array([curve(t,g,sig,s,h,c) for t in pos]))
    plot.set_ydata(np.array([curve3(t,g,sig,s,h,c) for t in pos]))
    fig.canvas.draw()

